# Run This Entire Notebook to Get Your Environment Setup, the Last Function is Just for Testing

In [70]:
from dotenv import load_dotenv
load_dotenv(".env")

import helpers.postgres_helper
import helpers.embedding_helper
import helpers.scraping_helpers
import helpers.LLM_helpers
import os
import importlib

In [71]:
importlib.reload(helpers.postgres_helper)
importlib.reload(helpers.embedding_helper)
importlib.reload(helpers.scraping_helpers)
importlib.reload(helpers.LLM_helpers)

<module 'helpers.LLM_helpers' from '/Users/williamloving/Documents/Githubs/ML-Project-Repo/test/helpers/LLM_helpers.py'>

In [72]:
pg_helper = helpers.postgres_helper.PostgresConnector()
scrape_helper = helpers.scraping_helpers.ScrapeHelpers()
embed_helper = helpers.embedding_helper.EmbeddingHelper()
LLM_helper = helpers.LLM_helpers.GeminiLLMHelper()

In [73]:
professors_and_abstracts = scrape_helper.read_json_to_dict("data/professors_papers_and_abstracts.json")

In [74]:
def populate_db(pg_helper, professors_and_abstracts):
    pg_helper.create_connection()
    for professor in professors_and_abstracts:
        professor_name = professor["Professor"]
        for paper in professor["Papers"]:
            paper_title = paper["title"]
            paper_abstract = paper["abstract"]
            paper_link = paper["paper_link"]
            if not paper_abstract or not paper_link or not paper_title:
                continue
            paper_embedding = embed_helper.create_embedding(paper_abstract)
            pg_helper.insert_professor_and_paper(professor_name, paper_title, paper_abstract, paper_embedding, paper_link)
            print(f"Inserted {paper_title} for {professor_name}")
    pg_helper.close_connection()

In [ ]:
populate_db(pg_helper, professors_and_abstracts)

In [76]:
def get_similar_papers(query, n=5):
    query_embedding = embed_helper.create_embedding(query)


    pg_helper.create_connection()
    similar_papers = pg_helper.find_similar_papers(query_embedding, 10)
    similar_papers.reverse()
    pg_helper.close_connection()

    return similar_papers

In [ ]:
query = "I like operating systems"


get_similar_papers(query, 10)

Connection to PostgreSQL DB successful
The connection to the PostgreSQL DB is closed


[{'id': 716,
  'title': 'Using LEGO MINDSTORMS NXT and LEJOS in an advanced software engineering course',
  'abstract': 'This paper describes the benefits of using LeJOS and the Lego Mindstorms NXT set for teaching advanced software development. While Lego Mindstorms has been used in introduction to computer science courses, it is not reported to be widely used in a simulated production environment requiring such things as threading, network communications, and the implementation of command protocols. Additionally, because the Mindstorms NXT system supports Bluetooth communications with multiple devices, it is possible to use this system as the basis for a complex, communicating system requiring multiple software artifacts on different machines.',
  'paper_link': 'https://www.semanticscholar.org/paper/b269756012f09938573caf76e8cfcf9f804afb16',
  'similarity': 0.7161985124290422,
  'author': 'Mark Sherriff'},
 {'id': 343,
  'title': 'Using LEGO MINDSTORMS NXT and LEJOS in an advanced so